# Topic modelling using gensim <a class="tocSkip">

<!--[kaggle.com/rahulvks/topic-modeling-using-gensim/notebook](https://www.kaggle.com/rahulvks/topic-modeling-using-gensim/notebook)-->

# Imports

## Import dependencies

In [1]:
%%bash
ls | grep .csv

emails.csv
emails.csv.zip


In [2]:
# built-in libs
import email

# processing libs
import pandas as pd

# display libs
from tqdm import tqdm_notebook

## Import data

In [3]:
emails_full_df = pd.read_csv('emails.csv', chunksize=10000)
emails_df = next(emails_full_df)

In [4]:
print(emails_df.shape)
emails_df.head()

(10000, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
emails_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
file       10000 non-null object
message    10000 non-null object
dtypes: object(2)
memory usage: 156.3+ KB


In [6]:
%time
messages_obj_lst = []
messages_str_lst = []

message_metadata = {}

for i in tqdm_notebook(range(emails_df.shape[0])):
    msg = email.message_from_string(emails_df.message[i])
    
    for msg_property in msg:
        if msg_property in message_metadata:
            message_metadata[msg_property][i] = msg[msg_property]
        else:
            message_metadata[msg_property] = ['N/A'] * emails_df.shape[0]
    
    payload = msg.get_payload() # decode=True
    
    messages_obj_lst.append(msg)
    messages_str_lst.append(payload) #.encode('utf-8').decode('unicode_escape')
    #except KeyboardInterrupt:
    #    break

print('messages_obj_lst size: %i' % len(messages_obj_lst))

CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 17.4 µs



messages_obj_lst size: 10000


In [7]:
# update dataframe object
# emails_df.rename(columns = {'message':'message_obj'}, inplace = True)
emails_df = emails_df.assign(message_obj = pd.Series(messages_obj_lst).values)
emails_df = emails_df.assign(payload     = pd.Series(messages_str_lst).values)

# print(emails_df.payload.str.contains(r'\\'))
emails_df['payload'] = emails_df.payload.str.replace(r'\n', '')

In [8]:
emails_df.head()

,file,message,message_obj,payload
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,"[Message-ID, Date, From, To, Subject, Mime-Ver...",Here is our forecast
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,"[Message-ID, Date, From, To, Subject, Mime-Ver...",Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,"[Message-ID, Date, From, To, Subject, Mime-Ver...",test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"[Message-ID, Date, From, To, Subject, Mime-Ver...","Randy, Can you send me a schedule of the salar..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,"[Message-ID, Date, From, To, Subject, Mime-Ver...",Let's shoot for Tuesday at 11:45.


In [9]:
for i in range(50):
    print(emails_df.message_obj[i]['Subject'])


Re:
Re: test

Re: Hello
Re: Hello

Re: PRC review - phone calls
Re: High Speed Internet Access
FW: fixed forward or other Collar floor gas price terms
Re: FW: fixed forward or other Collar floor gas price terms

Consolidated positions: Issues & To Do list
Consolidated positions: Issues & To Do list

Re: 2001 Margin Plan
Var, Reporting and Resources Meeting

Westgate
Meeting re: Storage Strategies in the West

Re: Not business related..
Re: Original Sept check/closing
San Juan Index
San Juan Index
Investment Structure
Investment Structure


Re: Gas Trading Vision meeting

Gas Physical/Financial Position
closing








Westgate Proforma-Phillip Allen.xls


Re: burnet

Re: Sept 1 Payment

Contact list for mid market



In [10]:
del messages_obj_lst
del messages_str_lst

emails_df.drop('message', axis=1, inplace=True)

In [11]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [12]:
train = emails_df[:7000]
test = emails_df[7000:]

In [13]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(emails_df.message_obj[row]['Subject']) # ' '.join(str(x))

trainheadlines = list(filter(None, trainheadlines))

[row for row in trainheadlines[:10]]

['Re:',
 'Re: test',
 'Re: Hello',
 'Re: Hello',
 'Re: PRC review - phone calls',
 'Re: High Speed Internet Access',
 'FW: fixed forward or other Collar floor gas price terms',
 'Re: FW: fixed forward or other Collar floor gas price terms',
 'Consolidated positions: Issues & To Do list',
 'Consolidated positions: Issues & To Do list']

In [14]:
# trainvect = CountVectorizer()
# Trainfeature = trainvect.fit_transform(trainheadlines)

In [15]:
# ####Detailed view of Document Count Matrix
# DTM_With_Colm = pd.DataFrame(Trainfeature.toarray(),columns= trainvect.get_feature_names())

In [16]:
# Trainfeature.shape

In [17]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [18]:
%time
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
#Our Document
trainheadlines

# list for tokenized documents in loop
texts = []

# loop through document list
for i in trainheadlines:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [19]:
%time
#generate LDA
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=1,chunksize=10000,update_every=1)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [20]:
%time
import pyLDAvis.gensim
print(ldamodel.print_topics(num_topics=10, num_words=3))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
[(0, '0.180*"re" + 0.014*"ng" + 0.011*"credit"'), (1, '0.034*"re" + 0.028*"fw" + 0.020*"call"'), (2, '0.020*"meet" + 0.019*"ga" + 0.017*"fw"'), (3, '0.102*"re" + 0.033*"ga" + 0.023*"trade"'), (4, '0.162*"re" + 0.014*"meet" + 0.010*"market"'), (5, '0.057*"enron" + 0.051*"re" + 0.030*"mention"'), (6, '0.220*"re" + 0.015*"10" + 0.014*"fw"'), (7, '0.093*"re" + 0.020*"request" + 0.018*"com"'), (8, '0.093*"re" + 0.019*"01" + 0.017*"enron"'), (9, '0.050*"re" + 0.020*"fw" + 0.016*"new"')]


In [21]:
ldamodel.print_topics(5)

[(0,
  '0.180*"re" + 0.014*"ng" + 0.011*"credit" + 0.010*"fw" + 0.008*"view" + 0.008*"ga" + 0.007*"extra" + 0.006*"s" + 0.006*"execut" + 0.006*"request"'),
 (1,
  '0.034*"re" + 0.028*"fw" + 0.020*"call" + 0.013*"ga" + 0.012*"confer" + 0.012*"30" + 0.012*"10" + 0.011*"western" + 0.010*"meet" + 0.010*"12"'),
 (6,
  '0.220*"re" + 0.015*"10" + 0.014*"fw" + 0.010*"weekli" + 0.010*"daili" + 0.009*"2001" + 0.009*"chart" + 0.009*"ng" + 0.008*"survey" + 0.008*"p"'),
 (2,
  '0.020*"meet" + 0.019*"ga" + 0.017*"fw" + 0.016*"10" + 0.015*"re" + 0.015*"2001" + 0.014*"enron" + 0.013*"01" + 0.013*"report" + 0.011*"market"'),
 (3,
  '0.102*"re" + 0.033*"ga" + 0.023*"trade" + 0.016*"fw" + 0.016*"2001" + 0.012*"daili" + 0.011*"west" + 0.009*"meet" + 0.009*"chart" + 0.009*"updat"')]

In [22]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
news = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary)



/usr/local/lib/python3.5/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [23]:
news

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      12.277516        1       1  0.105312 -0.066233
8      11.853916        1       2  0.014913 -0.033305
0      10.759757        1       3  0.118399  0.029564
3      10.732884        1       4  0.022781 -0.050311
4      10.686002        1       5  0.069748 -0.010095
2       9.830538        1       6 -0.127189 -0.038574
1       9.591447        1       7 -0.085652 -0.102694
7       9.246477        1       8  0.026889  0.101609
9       7.762489        1       9 -0.044289  0.115617
5       7.258973        1      10 -0.100912  0.054422, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
1155  Default  2281.000000            re  2281.000000  30.0000  30.0000
1259  Default   258.000000         enron   258.000000  29.0000  29.0000
1182  Default    57.000000       mention    57.000000  28.0000  28.0000
2252  Default   107.000000         trade   107.000000  27.0000  27.0000
1633  Default    68.000000          call    68.000000  26.0000  26.0000
2312  Default    92.000000            ng    92.000000  25.0000  25.0000
2030  Default    44.000000         notic    44.000000  24.0000  24.0000
602   Default   100.000000        access   100.000000  23.0000  23.0000
634   Default   265.000000            ga   265.000000  22.0000  22.0000
580   Default   166.000000            01   166.000000  21.0000  21.0000
782   Default    26.000000  transwestern    26.000000  20.0000  20.0000
12    Default    26.000000            co    26.000000  19.0000  19.0000
1000  Default    31.000000       pipelin    31.000000  18.0000  18.0000
126   Default   150.000000            10   150.000000  17.0000  17.0000
0     Default   165.000000          2001   165.000000  16.0000  16.0000
2095  Default   104.000000           com   104.000000  15.0000  15.0000
839   Default   290.000000            fw   290.000000  14.0000  14.0000
2053  Default   114.000000         daili   114.000000  13.0000  13.0000
1621  Default    57.000000             p    57.000000  12.0000  12.0000
1036  Default    71.000000         chart    71.000000  11.0000  11.0000
145   Default    46.000000        matric    46.000000  10.0000  10.0000
1044  Default    79.000000        market    79.000000   9.0000   9.0000
371   Default    51.000000             l    51.000000   8.0000   8.0000
1766  Default    45.000000        confer    45.000000   7.0000   7.0000
327   Default    34.000000         resum    34.000000   6.0000   6.0000
1580  Default   184.000000          meet   184.000000   5.0000   5.0000
216   Default    18.000000      attorney    18.000000   4.0000   4.0000
684   Default    54.000000          west    54.000000   3.0000   3.0000
1876  Default    19.000000      privileg    19.000000   2.0000   2.0000
1327  Default    67.000000           hot    67.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
1721  Topic10     3.773208          well     6.965258   2.0099  -6.0216
1708  Topic10     3.772984          what     7.425438   1.9459  -6.0217
2006  Topic10    12.055387       present    41.031551   1.3981  -4.8600
1915  Topic10     2.852616          rent     4.928737   2.0761  -6.3013
543   Topic10     3.825736           iso     7.986767   1.8869  -6.0078
2028  Topic10     3.773085          24th     7.940018   1.8789  -6.0217
580   Topic10    24.939896            01   166.811905   0.7225  -4.1331
2170  Topic10     4.693093       thought    12.062027   1.6790  -5.8035
2059  Topic10     3.615269         calif     7.993855   1.8294  -6.0644
618   Topic10     4.693508          note    13.223070   1.5871  -5.8034
502   Topic10     7.454184        impact    30.669857   1.2084  -5.3408
1155  Topic10    80.051476            re  2281.586426  -0.7270  -2.9669
2011  Topic10     5.598105       announc    18.712011   1.4162  -5.62

In [24]:
# %%bash
# nvidia-smi

---

# Bibliography


+ pip3 install stop-words
+ pip3 install pyLDAvis

---